# BigQuery - BQML

This notebook uses BigQuery ML to train a model to predict the value handwritten digits in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then evaluated and used for prediction using BigQuery directly.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-02.png">

---
## Setup

Parameters:

In [2]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [1]:
%%bigquery
CREATE OR REPLACE MODEL `digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `digits.digits_prepped`
WHERE splits = 'TRAIN'

""


Review the iterations from training:

In [14]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `digits.digits_lr`)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.195020,0.194182,0.2,5829
1,0,1,0.142442,0.140833,0.4,9967
2,0,2,0.088399,0.087190,0.8,9736
3,0,3,0.053124,0.053263,1.6,12358
4,0,4,0.033769,0.034647,3.2,7357
5,0,5,0.022698,0.023536,6.4,8301
6,0,6,0.016918,0.019726,12.8,7234
7,0,7,0.014701,0.016849,3.2,6475
8,0,8,0.013374,0.016339,6.4,7901
9,0,9,0.011472,0.014574,12.8,6993


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [15]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TEST'))

,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TEST,0.960701,0.960997,0.964072,0.960148,0.169828,0.998567
1,TRAIN,0.977487,0.977075,0.976760,0.977164,0.119165,0.999751


Review the confusion matrix for each split:

In [16]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,135,0,0,0,0,0,0,0,0,0
1,1,0,154,0,0,0,0,0,0,1,1
2,2,0,0,138,0,0,0,0,0,0,0
3,3,0,0,0,150,0,1,0,2,2,0
4,4,0,0,0,0,135,0,0,1,2,0
5,5,0,1,0,0,0,141,0,0,0,2
6,6,0,2,0,0,1,0,142,0,0,0
7,7,0,0,0,0,0,0,0,154,0,1
8,8,0,4,1,0,0,2,1,0,140,0
9,9,0,2,0,1,0,3,0,0,3,140


In [17]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TEST')
  );

,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,43,0,0,0,0,0,0,0,0,0
1,1,0,25,0,0,0,0,0,0,1,0
2,2,0,0,38,0,0,0,0,1,0,0
3,3,0,0,1,25,0,1,0,0,1,0
4,4,0,0,0,0,42,0,0,0,1,0
5,5,0,0,0,0,0,37,1,0,0,0
6,6,0,1,0,0,0,0,35,0,0,0
7,7,0,0,0,0,0,0,0,24,0,0
8,8,0,2,0,0,0,0,0,0,24,0
9,9,0,0,0,0,1,0,0,0,1,29


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [18]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits='TEST')
  )

Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [19]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'SPLITS'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [20]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,7,7,"[{'label': 7, 'prob': 0.9935705821896978}, {'l...",TEST
1,6,6,"[{'label': 6, 'prob': 0.9814892663856343}, {'l...",TEST
2,6,6,"[{'label': 6, 'prob': 0.9946601009802911}, {'l...",TEST
3,5,5,"[{'label': 5, 'prob': 0.7394754219029634}, {'l...",TEST
4,9,9,"[{'label': 9, 'prob': 0.9096139449606738}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [21]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 1.8939019560982916e-05},
  {'label': 1, 'prob': 0.0001175955035936026},
  {'label': 2, 'prob': 3.630831549124425e-05},
  {'label': 3, 'prob': 0.00028352950504693285},
  {'label': 4, 'prob': 0.0002572503372567677},
  {'label': 5, 'prob': 9.825240153115115e-05},
  {'label': 6, 'prob': 6.228722503524457e-06},
  {'label': 7, 'prob': 0.9935705821896978},
  {'label': 8, 'prob': 0.00012288049359936875},
  {'label': 9, 'prob': 0.005488433511718718}]]

---
## Remove Resources
- delete model `<PROJECT_ID>.digits.digits_lr`

In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
# NOTE: This is an input for Notebook 03 - BigQuery - BQML Online Predictions 
bq.delete_model(PROJECT_ID+'.digits.digits_lr')